### 1. 安装HuggingFace 并下载模型到本地

In [1]:
!pip install huggingface-hub -Uqq
!pip install -U sagemaker

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tokenizers 0.14.1 requires huggingface_hub<0.18,>=0.16.4, but you have huggingface-hub 0.19.4 which is incompatible.


In [ ]:
# stella-en

from huggingface_hub import snapshot_download
from pathlib import Path

local_model_path_name = "./infrgad_stella_large_zh_v2"
model_hf_name = "infgrad/stella-large-zh-v2"
model_name = model_hf_name.split('/')[-1]
commit_hash = "f11c871ed1df5768a73e5bfafcec88b122648905"

local_model_path = Path(local_model_path_name)
local_model_path.mkdir(exist_ok=True)
model_name = model_name
commit_hash = commit_hash

In [34]:
# # stella-zh

# from huggingface_hub import snapshot_download
# from pathlib import Path

# local_model_path_name = "./infrgad_stella_large_zh_v2"
# model_hf_name = "infgrad/stella-large-zh-v2"
# model_name = model_hf_name.split('/')[-1]
# commit_hash = "f11c871ed1df5768a73e5bfafcec88b122648905"

# local_model_path = Path(local_model_path_name)
# local_model_path.mkdir(exist_ok=True)
# model_name = model_name
# commit_hash = commit_hash

In [3]:
snapshot_download(repo_id=model_hf_name, revision=commit_hash, cache_dir=local_model_path)

Fetching 8 files:   0%|          | 0/8 [00:00<?, ?it/s]

.gitattributes:   0%|          | 0.00/1.52k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/315 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/40.2k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/110k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/439k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/882 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/652M [00:00<?, ?B/s]

'infrgad_stella_large_zh_v2/models--infgrad--stella-large-zh-v2/snapshots/f11c871ed1df5768a73e5bfafcec88b122648905'

In [5]:
import os
from dotenv import load_dotenv
load_dotenv()
name="ATL_UW2"
profile_name = os.environ[f"{name}"]
%env AWS_DEFAULT_PROFILE = {profile_name}
s3_bucket_name=os.environ[f"{name}_S3_BUCKET_NAME"]
role=os.environ[f"{name}_ROLE"]
s3_bucket_name

env: AWS_DEFAULT_PROFILE=atl-uw2


'aws-gcr-csdc-atl-exp-us-west-2'

### 2. 把模型拷贝到S3为后续部署做准备

In [6]:
import sagemaker
from sagemaker import image_uris
import boto3
import os
import time
import json

#role = sagemaker.get_execution_role()  # execution role for the endpoint
sess = sagemaker.session.Session()  # sagemaker session for interacting with different AWS APIs
# bucket = sess.default_bucket()  # bucket to house artifacts
bucket = s3_bucket_name  # bucket to house artifacts

region = sess._region_name
account_id = sess.account_id()

s3_client = boto3.client("s3")
sm_client = boto3.client("sagemaker")
smr_client = boto3.client("sagemaker-runtime")

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ubuntu/.config/sagemaker/config.yaml


In [7]:
s3_model_prefix = f"aigc-embedding-models/{model_name}"  # folder where model checkpoint will go
model_snapshot_path = list(local_model_path.glob("**/snapshots/*"))[0]
s3_code_prefix = f"aigc-embedding-models/{model_name}_deploy_code"
print(f"s3_code_prefix: {s3_code_prefix}")
print(f"model_snapshot_path: {model_snapshot_path}")

s3_code_prefix: aigc-embedding-models/stella-large-zh-v2_deploy_code
model_snapshot_path: infrgad_stella_large_zh_v2/models--infgrad--stella-large-zh-v2/snapshots/f11c871ed1df5768a73e5bfafcec88b122648905


In [8]:
!aws s3 cp --recursive {model_snapshot_path} s3://{bucket}/{s3_model_prefix}

upload: infrgad_stella_large_zh_v2/models--infgrad--stella-large-zh-v2/snapshots/f11c871ed1df5768a73e5bfafcec88b122648905/special_tokens_map.json to s3://aws-gcr-csdc-atl-exp-us-west-2/aigc-embedding-models/stella-large-zh-v2/special_tokens_map.json
upload: infrgad_stella_large_zh_v2/models--infgrad--stella-large-zh-v2/snapshots/f11c871ed1df5768a73e5bfafcec88b122648905/.gitattributes to s3://aws-gcr-csdc-atl-exp-us-west-2/aigc-embedding-models/stella-large-zh-v2/.gitattributes
upload: infrgad_stella_large_zh_v2/models--infgrad--stella-large-zh-v2/snapshots/f11c871ed1df5768a73e5bfafcec88b122648905/README.md to s3://aws-gcr-csdc-atl-exp-us-west-2/aigc-embedding-models/stella-large-zh-v2/README.md
upload: infrgad_stella_large_zh_v2/models--infgrad--stella-large-zh-v2/snapshots/f11c871ed1df5768a73e5bfafcec88b122648905/config.json to s3://aws-gcr-csdc-atl-exp-us-west-2/aigc-embedding-models/stella-large-zh-v2/config.json
upload: infrgad_stella_large_zh_v2/models--infgrad--stella-large-zh-v2

### 3. 模型部署准备（entrypoint脚本，容器镜像，服务配置）

In [9]:
inference_image_uri = (
    f"763104351884.dkr.ecr.{region}.amazonaws.com/djl-inference:0.21.0-deepspeed0.8.3-cu117"
)

#中国区需要替换为下面的image_uri
# inference_image_uri = (
#     f"727897471807.dkr.ecr.{region}.amazonaws.com.cn/djl-inference:0.21.0-deepspeed0.8.3-cu117"
# )

print(f"Image going to be used is ---- > {inference_image_uri}")

Image going to be used is ---- > 763104351884.dkr.ecr.us-west-2.amazonaws.com/djl-inference:0.21.0-deepspeed0.8.3-cu117


In [10]:
local_code_dir = s3_code_prefix.split('/')[-1]
!mkdir -p {local_code_dir}

In [11]:
%%writefile {local_code_dir}/model.py
from djl_python import Input, Output
import torch
import logging
import math
import os
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer, AutoModel

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(f'--device={device}')


def load_model(properties):
    tensor_parallel = properties["tensor_parallel_degree"]
    model_location = properties['model_dir']
    if "model_id" in properties:
        model_location = properties['model_id']
    logging.info(f"Loading model in {model_location}")
    
    tokenizer = AutoTokenizer.from_pretrained(model_location, trust_remote_code=True)
    # tokenizer.padding_side = 'right'
    model = AutoModel.from_pretrained(
        model_location, 
        # device_map="balanced_low_0", 
        trust_remote_code=True
    ).half()
    # load the model on GPU
    model.to(device) 
    model.requires_grad_(False)
    model.eval()
    
    return model, tokenizer


model = None
tokenizer = None
generator = None

def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0].to(device) #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float().to(device)
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)


def handle(inputs: Input):
    global model, tokenizer
    if not model:
        model, tokenizer = load_model(inputs.get_properties())

    if inputs.is_empty():
        return None
    data = inputs.get_as_json()
    
    input_sentences = data["inputs"]
    logging.info(f"inputs: {input_sentences}")
    
    # encoded_input = tokenizer(input_sentences, padding=True, truncation=True, return_tensors='pt').to(device)
    encoded_input = tokenizer(input_sentences, padding="longest", max_length=1024, truncation=True, return_tensors='pt').to(device)
    # Compute token embeddings
    with torch.no_grad():
        attention_mask = encoded_input["attention_mask"]
        model_output = model(**encoded_input)
        last_hidden = model_output.last_hidden_state.masked_fill(~attention_mask[..., None].bool(), 0.0)
        sentence_embeddings = last_hidden.sum(dim=1) / attention_mask.sum(dim=1)[..., None]

    # Perform pooling. In this case, max pooling.
    # sentence_embeddings = model_output.cpu().numpy()
    sentence_embeddings = torch.nn.functional.normalize(sentence_embeddings, p=2, dim=1).cpu().numpy()

#     # preprocess
#     input_ids = tokenizer(input_sentences, return_tensors="pt").input_ids
#     # pass inputs with all kwargs in data
#     if params is not None:
#         outputs = model.generate(input_ids, **params)
#     else:
#         outputs = model.generate(input_ids)

#     # postprocess the prediction
#     prediction = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    result = {"sentence_embeddings": sentence_embeddings}
    return Output().add_as_json(result)

Writing stella-large-zh-v2_deploy_code/model.py


In [12]:
s3_path = f"s3://{bucket}/{s3_model_prefix}/"
print(f"option.s3url ==> {s3_path}")

option.s3url ==> s3://aws-gcr-csdc-atl-exp-us-west-2/aigc-embedding-models/stella-large-zh-v2/


#### Note: option.s3url 需要按照自己的账号进行修改, 可以拷贝上一个cell的输出

In [26]:
%%writefile {local_code_dir}/serving.properties
engine=Python
option.tensor_parallel_degree=1
# option.s3url = fs3://sagemaker-us-west-2-316327952690/LLM-RAG/workshop/buffer-embedding-002-model/
option.s3url = S3PATH

Overwriting stella-large-zh-v2_deploy_code/serving.properties


In [27]:
!sed -i "s|option.s3url = S3PATH|option.s3url = {s3_path}|" {local_code_dir}/serving.properties

In [28]:
!rm s2e_model.tar.gz
!cd {local_code_dir} && rm -rf ".ipynb_checkpoints"
!tar czvf s2e_model.tar.gz {local_code_dir}

stella-large-zh-v2_deploy_code/
stella-large-zh-v2_deploy_code/model.py
stella-large-zh-v2_deploy_code/serving.properties


In [29]:
s3_code_artifact = sess.upload_data("s2e_model.tar.gz", bucket, s3_code_prefix)
print(f"S3 Code or Model tar ball uploaded to --- > {s3_code_artifact}")

S3 Code or Model tar ball uploaded to --- > s3://aws-gcr-csdc-atl-exp-us-west-2/aigc-embedding-models/stella-large-zh-v2_deploy_code/s2e_model.tar.gz


### 4. 创建模型 & 创建endpoint

In [37]:
from sagemaker.utils import name_from_base
import boto3

model_name_ep = name_from_base(f"{model_name}").replace('.','-').replace('_','-')# name_from_base("st-paraphrase-mpnet-base-v2") Note: Need to specify model_name
print(model_name_ep)
print(f"Image going to be used is ---- > {inference_image_uri}")

create_model_response = sm_client.create_model(
    ModelName=model_name_ep,
    ExecutionRoleArn=role,
    PrimaryContainer={
        "Image": inference_image_uri,
        "ModelDataUrl": s3_code_artifact
    },
    
)
model_arn = create_model_response["ModelArn"]

print(f"Created Model: {model_arn}")

stella-large-zh-v2-2023-11-20-03-07-47-287
Image going to be used is ---- > 763104351884.dkr.ecr.us-west-2.amazonaws.com/djl-inference:0.21.0-deepspeed0.8.3-cu117


Created Model: arn:aws:sagemaker:us-west-2:316327952690:model/stella-large-zh-v2-2023-11-20-03-07-47-287


#### 推理机型选择 (https://aws.amazon.com/cn/sagemaker/pricing/)
- GPU
  + ml.g4dn.xlarge 按需价格 0.526 USD/Hour
- CPU
  + ml.c5.xlarge   按需价格 0.204 USD/Hour

In [41]:
endpoint_config_name = f"{model_name_ep}-config"
endpoint_name = f"{model_name_ep}-endpoint"

endpoint_config_response = sm_client.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    ProductionVariants=[
        {
            "VariantName": "variant1",
            "ModelName": model_name_ep,
            "InstanceType": "ml.g5.2xlarge",
            "InitialInstanceCount": 1,
            # "VolumeSizeInGB" : 400,
            # "ModelDataDownloadTimeoutInSeconds": 2400,
            "ContainerStartupHealthCheckTimeoutInSeconds": 15*60,
        },
    ],
)
endpoint_config_response

{'EndpointConfigArn': 'arn:aws:sagemaker:us-west-2:316327952690:endpoint-config/stella-large-zh-v2-2023-11-20-03-07-47-287-config',
 'ResponseMetadata': {'RequestId': '6f208736-c2fa-4310-8988-ffb56c87f089',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '6f208736-c2fa-4310-8988-ffb56c87f089',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '130',
   'date': 'Mon, 20 Nov 2023 03:08:26 GMT'},
  'RetryAttempts': 0}}

In [42]:
endpoint_name

'stella-large-zh-v2-2023-11-20-03-07-47-287-endpoint'

In [43]:
from datetime import datetime

currentDay = datetime.now().day
currentMonth = datetime.now().month
currentYear = datetime.now().year

current_time = f"{currentYear}{currentMonth}{currentDay}"
current_time


tag=f"{current_time}-{commit_hash}"

create_endpoint_response = sm_client.create_endpoint(
    EndpointName=f"{endpoint_name}", EndpointConfigName=endpoint_config_name, Tags=[{"Key":"version", "Value":tag}],
)
print(f"Created Endpoint: {create_endpoint_response['EndpointArn']}")

Created Endpoint: arn:aws:sagemaker:us-west-2:316327952690:endpoint/stella-large-zh-v2-2023-11-20-03-07-47-287-endpoint


In [44]:
import time

resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
status = resp["EndpointStatus"]
print("Status: " + status)

while status == "Creating":
    time.sleep(60)
    resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
    status = resp["EndpointStatus"]
    print("Status: " + status)

print("Arn: " + resp["EndpointArn"])
print("Status: " + status)

Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: InService
Arn: arn:aws:sagemaker:us-west-2:316327952690:endpoint/stella-large-zh-v2-2023-11-20-03-07-47-287-endpoint
Status: InService


### 5. 模型测试

In [45]:
def get_vector_by_sm_endpoint(questions, sm_client, endpoint_name):
    parameters = {
        # "early_stopping": True,
        # "length_penalty": 2.0,
        "max_new_tokens": 50,
        "temperature": 0,
        "min_length": 10,
        "no_repeat_ngram_size": 2,
    }

    response_model = sm_client.invoke_endpoint(
        EndpointName=endpoint_name,
        Body=json.dumps(
            {
                "inputs": questions,
            }
        ),
        ContentType="application/json",
    )
    json_str = response_model['Body'].read().decode('utf8')
    json_obj = json.loads(json_str)
    embeddings = json_obj['sentence_embeddings']
    return embeddings

In [46]:
# test for zh endpoint
import numpy as np

endpoint_name = "stella-large-zh-v2-2023-11-20-03-07-47-287-endpoint"


cand1 = []
cand1.append("AWS提供的关键服务有哪些，以支持不同类型的企业需求和工作负载？")
cand1.append("Amazon SageMaker提供哪些关键功能，以支持企业在机器学习领域的需求和挑战？")
cand2 = []
cand2.append("AWS的全球基础架构是如何支持高可靠性和可扩展性，以满足企业的云计算需求？")
cand2.append("Amazon SageMaker如何简化机器学习模型的开发、训练和部署过程，以提高企业的生产效率和创新能力？")

cand = cand1 + cand2

res = get_vector_by_sm_endpoint(cand, smr_client, endpoint_name)

cand1_embed = res[0:2]
cand2_embed = res[2:]

results = {}

for idx1, cand1_score in enumerate(cand1_embed):
    for idx2, cand2_score in enumerate(cand2_embed):
        # print(cand2_score)
        print(f"{cand1[idx1]}")
        print(f"{cand2[idx2]}") 
        print(f"score: {np.dot(np.array(cand1_score), np.array(cand2_score).T)}")

AWS提供的关键服务有哪些，以支持不同类型的企业需求和工作负载？
AWS的全球基础架构是如何支持高可靠性和可扩展性，以满足企业的云计算需求？
score: 0.7902514349186731
AWS提供的关键服务有哪些，以支持不同类型的企业需求和工作负载？
Amazon SageMaker如何简化机器学习模型的开发、训练和部署过程，以提高企业的生产效率和创新能力？
score: 0.5964026839469625
Amazon SageMaker提供哪些关键功能，以支持企业在机器学习领域的需求和挑战？
AWS的全球基础架构是如何支持高可靠性和可扩展性，以满足企业的云计算需求？
score: 0.6569513314645974
Amazon SageMaker提供哪些关键功能，以支持企业在机器学习领域的需求和挑战？
Amazon SageMaker如何简化机器学习模型的开发、训练和部署过程，以提高企业的生产效率和创新能力？
score: 0.8153492034732608


In [40]:
# test for en endpoint
import numpy as np

endpoint_name = "bge-large-en-v1-5-2023-11-15-06-19-29-526-endpoint"


cand1 = []
cand1.append("What are some key advantages of adopting AWS for cloud computing compared to traditional on-premises infrastructure?")
cand1.append("What are the key advantages of using Amazon SageMaker for machine learning compared to traditional ML development workflows?")
cand2 = []
cand2.append("Can you explain the main benefits that businesses can gain by leveraging AWS services for their cloud computing needs?")
cand2.append("How does Amazon SageMaker simplify the process of building, training, and deploying machine learning models compared to traditional approaches?")

cand = cand1 + cand2

res = get_vector_by_sm_endpoint(cand, smr_client, endpoint_name)

cand1_embed = res[0:2]
cand2_embed = res[2:]

results = {}

for idx1, cand1_score in enumerate(cand1_embed):
    for idx2, cand2_score in enumerate(cand2_embed):
        # print(cand2_score)
        print(f"{cand1[idx1]}")
        print(f"{cand2[idx2]}") 
        print(f"score: {np.dot(np.array(cand1_score), np.array(cand2_score).T)}")

What are some key advantages of adopting AWS for cloud computing compared to traditional on-premises infrastructure?
Can you explain the main benefits that businesses can gain by leveraging AWS services for their cloud computing needs?
score: 0.8515167698601687
What are some key advantages of adopting AWS for cloud computing compared to traditional on-premises infrastructure?
How does Amazon SageMaker simplify the process of building, training, and deploying machine learning models compared to traditional approaches?
score: 0.5905170103072663
What are the key advantages of using Amazon SageMaker for machine learning compared to traditional ML development workflows?
Can you explain the main benefits that businesses can gain by leveraging AWS services for their cloud computing needs?
score: 0.5681599540677098
What are the key advantages of using Amazon SageMaker for machine learning compared to traditional ML development workflows?
How does Amazon SageMaker simplify the process of buildi

In [25]:
prompts1 = """请问AWS Clean Rooms是多方都会收费吗？"""

res = get_vector_by_sm_endpoint([prompts1], smr_client, endpoint_name)

In [23]:
prompts1 = """What is the purpose of Amazon S3 in AWS, and how is it typically used?"""

res = get_vector_by_sm_endpoint([prompts1], smr_client, endpoint_name)

In [26]:
res

[[0.0284881591796875,
  0.00933074951171875,
  -0.0020427703857421875,
  -0.013763427734375,
  0.0305328369140625,
  0.049560546875,
  -0.052459716796875,
  -0.039093017578125,
  -0.02337646484375,
  -0.0163726806640625,
  0.025054931640625,
  0.0250091552734375,
  -0.01207733154296875,
  -0.0123443603515625,
  -0.016876220703125,
  0.0016727447509765625,
  0.07147216796875,
  -0.007404327392578125,
  0.019775390625,
  -0.050018310546875,
  0.0016002655029296875,
  -0.0083465576171875,
  -0.005664825439453125,
  -0.0308074951171875,
  -0.033355712890625,
  0.0217742919921875,
  -0.0012178421020507812,
  -0.0186920166015625,
  -0.004138946533203125,
  -0.01418304443359375,
  -0.07318115234375,
  -0.0193328857421875,
  0.0055999755859375,
  0.00785064697265625,
  -0.01187896728515625,
  -0.0022487640380859375,
  -0.0285491943359375,
  0.01004791259765625,
  0.016754150390625,
  -0.048583984375,
  -0.00510406494140625,
  0.0197906494140625,
  0.0015707015991210938,
  -0.0213165283203125,
